In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
DATASET_START_DATE = '2022-03-04' 
NFT_COLLECTION = 'Sandbox' #options: 'Decentraland', 'Sandbox'
FUTURE_DATAFRAME_DAYS = 30


# eth_df = yf.download('ETH-USD',DATASET_START_DATE, today)

eth_df=pd.read_csv("/Users/School/Documents/MDASC/stat7008/Group Project/NFTs_WEB_APP/nft_price.csv")
eth_df.head()
#NFTs avg price prediction
eth_df['Close'] = eth_df['Sandbox'].shift(1)
eth_df = eth_df[(eth_df['Date'] > DATASET_START_DATE)]
eth_df


,Date,Decentraland,Sandbox,Close
119,2022-03-05,"$12,626.46","$8,922.20","$8,799.84"
120,2022-03-06,"$14,592.72","$8,504.98","$8,922.20"
121,2022-03-07,"$11,831.43","$7,737.87","$8,504.98"
122,2022-03-08,"$12,332.03","$8,538.25","$7,737.87"
123,2022-03-09,"$12,296.66","$8,524.53","$8,538.25"
124,2022-03-10,"$11,905.19","$8,468.20","$8,524.53"
125,2022-03-11,"$11,016.24","$7,814.54","$8,468.20"
126,2022-03-12,"$12,572.84","$8,225.58","$7,814.54"
127,2022-03-13,"$9,139.64","$8,359.37","$8,225.58"
128,2022-03-14,"$13,047.89","$8,385.90","$8,359.37"


# New section

In [4]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 119 to 151
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          33 non-null     object 
 1   Decentraland  33 non-null     float64
 2   Sandbox       33 non-null     float64
 3   Close         33 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.3+ KB


In [5]:
eth_df.isnull().sum()

Date            0
Decentraland    0
Sandbox         0
Close           0
dtype: int64

In [6]:
eth_df.columns

Index(['Date', 'Decentraland', 'Sandbox', 'Close'], dtype='object')

In [7]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['index', 'Date', 'Decentraland', 'Sandbox', 'Close'], dtype='object')

In [8]:
df = eth_df[["Date", NFT_COLLECTION]]

new_names = {
    "Date": "ds", 
    NFT_COLLECTION: "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
28,2022-04-02,"$8,164.41"
29,2022-04-03,"$7,772.34"
30,2022-04-04,"$7,348.95"
31,2022-04-05,"$7,064.86"
32,2022-04-06,"$7,150.04"


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of avg price of "+ NFT_COLLECTION +"'s NFTs ",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    # seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = 0.444831
Iteration  1. Log joint probability =    33.7976. Improved by 33.3528.
Iteration  2. Log joint probability =    56.5956. Improved by 22.798.
Iteration  3. Log joint probability =    72.0115. Improved by 15.4159.
Iteration  4. Log joint probability =    72.3887. Improved by 0.377201.
Iteration  5. Log joint probability =    73.8594. Improved by 1.4707.
Iteration  6. Log joint probability =    73.8799. Improved by 0.0204889.
Iteration  7. Log joint probability =    73.8856. Improved by 0.0057112.
Iteration  8. Log joint probability =    73.9163. Improved by 0.0307028.
Iteration  9. Log joint probability =    73.9606. Improved by 0.0443081.
Iteration 10. Log joint probability =    73.9809. Improved by 0.0202977.
Iteration 11. Log joint probability =    74.0284. Improved by 0.0474476.
Iteration 12. Log joint probability =    74.0955. Improved by 0.0671012.
Iteration 13. Log joint probability =    74.1145. Improved by 0.0190692.
Iteration 14. Log joi

In [12]:
future = m.make_future_dataframe(periods = FUTURE_DATAFRAME_DAYS)
future.tail()

,ds
58,2022-05-02
59,2022-05-03
60,2022-05-04
61,2022-05-05
62,2022-05-06


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
58,2022-05-02,"$5,346.90","$4,513.83","$6,171.92"
59,2022-05-03,"$5,410.33","$4,592.06","$6,296.64"
60,2022-05-04,"$5,552.58","$4,668.84","$6,424.15"
61,2022-05-05,"$5,676.08","$4,717.72","$6,639.04"
62,2022-05-06,"$5,696.19","$4,686.17","$6,724.63"


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

6558.206000599778

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)